# 实践运行_布林带策略

In [1]:
from tick_trade_api import TradeAPI
# 输入指定账户
trade_api=TradeAPI('79855040',order_policy=MarketPolicy)

source_code1 = """

# 初始化函数
def init(context):
    #设置Boll模型参数
    context.timeperiod = 16 
    context.nbdevup = 2 
    context.nbdevdn = 2    
    # 运作周期
    context.runstep = 3
    # 设置操作标的：
    context.stockpool_x = ['002633.SZ']

     # 设置初识持仓
    set_holding_stocks(dict(zip(context.stockpool_x,[0]*len(context.stockpool_x))))
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value
    stocklist = list(context.portfolio.stock_account.positions)
    print('当前持仓市值:', market_value, '持仓股票列表', stocklist, '当前账户总资金:', int(total_value))
    print()
 
def handle_bar(context, bar_dict):
    # 获取当前分钟
    time = get_datetime().strftime('%M')
    # 循环股票
    for stock in context.stockpool_x:
        #print('当前股票是：', stock)
        # 获取数据
        df = history(stock, ['close','high','low'], 2400,'1m', True, 'pre', is_panel=1)
        # 提取数据
        close_price = df['close']
        high_price = df['high']
        low_price = df['low']
        #print('=====',df['close'].iloc[-1], type(df['close']))
        new_price = df['close'].iloc[-1]
        
        # 导入Python-talib库
        import talib
        import numpy as np
        # 计算指标，获取BOLL值
        upperband, middleband, lowerband = talib.BBANDS(close_price.values, timeperiod=context.timeperiod, nbdevup=context.nbdevup, nbdevdn=context.nbdevdn)
        T_value = upperband-lowerband
        T_value = T_value[19:]
        mean = np.mean(T_value)
        new = T_value[-1]
        
        if T_value[-2] >T_value[-1] and new<mean and new_price<lowerband[-1]:
            print('当前处于相对低点{}'.format(new_price))
            order_percent(stock,0.5)
        elif T_value[-2] >T_value[-1] and new<mean and new_price>upperband[-1]:
            print('当前处于相对高点{}'.format(new_price))
            order_percent(stock,-0.5)
            
# 收盘后运行函数,用于储存自定义参数、全局变量,执行盘后选股等
def after_trading(context):
    market_value = context.portfolio.stock_account.market_value
    total_value = context.portfolio.stock_account.total_value
    stocklist = list(context.portfolio.stock_account.positions)
    print('当前持仓市值:', market_value, '持仓股票列表', stocklist, '当前账户总资金:', int(total_value))
    log.info('一天结束')            

"""


# 系统重置
try:
    import shutil
    shutil.rmtree('persist/研究环境策略')
except:
    pass

# 模拟运行
#rtrade = research_trade('研究环境策略', source_code1, capital_base=float(50000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易


# 回测运行
# research_strategy(source_code, start_date=None, end_date=None, capital_base=100000, frequency='DAILY', stock_market='STOCK', benchmark=None)
#research_strategy(source_code1, start_date='20180901', end_date='20220607', capital_base=float(50000), frequency='MINUTE', stock_market='STOCK', benchmark=None)




In [2]:
# 模拟运行
rtrade = research_trade('研究环境策略', source_code1, capital_base=float(50000), frequency='MINUTE', trade_api=trade_api)#trade_api可传入List[TradeAPI]，实现多账户交易


2022-06-15 14:40:07.344907 - INFO - 开始research_trade运行...
2022-06-15 00:00:00.000000 - INFO - 初始化002633.SZ持仓0股
2022-06-15 00:00:00.000000 - INFO - 当前持仓市值: 0.0 持仓股票列表 ['002633.SZ'] 当前账户总资金: 50000
2022-06-15 00:00:00.000000 - INFO - 
2022-06-15 15:30:00.000000 - INFO - 当前持仓市值: 0 持仓股票列表 [] 当前账户总资金: 50000
2022-06-15 15:30:00.000000 - INFO - 一天结束
2022-06-15 15:30:00.000000 - INFO - 结束research_trade运行...
